In [61]:
import _pickle as cPickle
import pickle
import pandas as pd
import os
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA as sklearnPCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn import cluster as Kcluster, metrics as Met ,preprocessing as preprocessing
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn import metrics, cross_validation
import matplotlib.pyplot as plt
from tabulate import tabulate

# Read the data, flatten and subsample the classes


In [80]:
#Create Sample Set . Holds the sample IDs and the labels under the column [accession,label]
SAMPLES = pd.read_csv(r"C:\Users\ajinkuriakose\Desktop\Computational Biology\Project\p1_train.csv", sep=',', error_bad_lines=False)
labels = ['CEU','FIN','GBR','TSI','YRI']
sample_set = pd.DataFrame()

#Equally Subsample from each of the classses
for lab in labels:
    SAMPLES_FILETERED = SAMPLES[SAMPLES.label == lab].sample(n=50)
    sample_set =sample_set.append(SAMPLES_FILETERED,ignore_index=False)
    
# pickle.dump(sample_set,open("sample300_ids.pkl",'wb'))

In [ ]:
# '''
#Create Data Set
# sample_set= pickle.load(open("sample75_ids.pkl",'rb'))
sample_set_id = sample_set['accession']
sample_data = pd.DataFrame()
root = r"C:\Users\ajinkuriakose\Desktop\Computational Biology\train"
for sampleid in sample_set_id:
    try:
        #ambig_info_df = pd.read_csv(root+"\\"+sampleid+ '\\bias\\aux_info\\ambig_info.tsv',sep='\t',error_bad_lines=False)
        quantified_data = pd.read_csv(root+"\\"+sampleid+"\\bias\\quant1.sf",sep='\t',error_bad_lines=False)
        #quantified_data = pd.DataFrame(pd.concat([quantified_data, ambig_info_df], axis=1).set_index('Name').stack()).transpose()
        quantified_data = pd.DataFrame(quantified_data.set_index('Name').stack()).transpose()
        sample_data = sample_data.append(quantified_data)
    except KeyboardInterrupt:
        break 
#     sample_data.drop('Length', axis=1, level=1, inplace=True)
#     pickle.dump(sample_data, open("data_ambig_300.pkl", 'wb'))
# '''

Optionally Load the data from data that was pickled previously

In [30]:
sample_data= pickle.load(open("data_ambig_250.pkl",'rb')) #data_150.pkl
sample_data= sample_data.dropna() 
sample_data.shape

(150, 797296)

# Data Format

In [4]:
sample_data.head()

Name ENST00000456328.2                                            \
       EffectiveLength       TPM NumReads UniqueCount AmbigCount   
0              1449.77  0.085384  2.14102         1.0        6.0   
0              1503.66  0.000000  0.00000         0.0        4.0   
0              1501.26  0.083892  3.44368         1.0        3.0   
0              1454.66  0.000000  0.00000         0.0        2.0   
0              1408.86  0.000000  0.00000         0.0        5.0   

Name ENST00000450305.2                                         ...      \
       EffectiveLength  TPM NumReads UniqueCount AmbigCount    ...       
0              486.623  0.0      0.0         0.0        1.0    ...       
0              498.230  0.0      0.0         0.0        1.0    ...       
0              169.000  0.0      0.0         0.0        0.0    ...       
0              184.000  0.0      0.0         0.0        0.0    ...       
0              459.052  0.0      0.0         0.0        1.0    ...       

Name ENST00000387460.2                                            \
       EffectiveLength       TPM NumReads UniqueCount AmbigCount   
0                 21.0   421.238    153.0       153.0        0.0   
0                 22.0   384.092    138.0       138.0        0.0   
0                 19.0   119.341     62.0        62.0        0.0   
0                 20.0   331.118    151.0       151.0        0.0   
0                 22.0  1323.940    674.0       674.0        0.0   

Name ENST00000387461.2                                           
       EffectiveLength      TPM NumReads UniqueCount AmbigCount  
0                 21.0  3350.63   1217.0      1217.0        0.0  
0                 22.0  2148.69    772.0       772.0        0.0  
0                 20.0  2333.32   1276.0      1276.0        0.0  
0                 21.0  4394.02   2104.0      2104.0        0.0  
0                 22.0  3628.06   1847.0      1847.0        0.0  

[5 rows x 996620 columns]

In [11]:
class predictorROC:
    
    #Yreal is the real outcome value, Yscore is the caluclated value
    #threshold is the threshold value to calculate the specificity and sensitivity
    
    def draw_roc(self, Yreal, Yscore):
        fpr = []
        tpr = []
        roc_auc = []
        fpr, tpr, _ = roc_curve(Yreal, Yscore)
        roc_auc = auc(fpr, tpr)

        plt.figure()
        lw = 2
        plt.plot(fpr, tpr, color='darkorange',
                 lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

# Dimensionality Reduction Using PCA

In [5]:
def get_eigenvalues(data):
    U, V, W = np.linalg.svd(data, full_matrices=False)
    return U,V

def perf_pca(data):
    #U, V, W = np.linalg.svd(data, full_matrices=False)
    #newV = [x for x in V if x >= 1]
    # find the number of principal components.
    # select the components with eigenvalue greater than 1.
    #dim = len(newV)
    sk_pca = sklearnPCA(n_components=100)
    pca = sk_pca.fit_transform(data)
    return pca

In [2]:
#Unable to do normalization for large inputs due to memory issues.
# Normalize the data
sample_norm_data = preprocessing.scale(np.array(sample_data))
# Compute the correlation matrix
sample_corr_data = np.corrcoef(sample_norm_data, rowvar=False)
#pca_sample = perf_pca(sample_corr_data)

In [82]:
pca=perf_pca(sample_data)
# pca = pickle.load(open("pca_ambig_250.pkl",'rb'))
pca.shape

(250, 250)

In [ ]:
'''
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=75)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)

#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)

#
from sklearn.linear_model import LogisticRegression
logm = LogisticRegression()
logm.fit(X_train, y_train)
predictions = logm.predict(X_test)
'''

# Train the Model using LogisticRegression and 5-fold cross validation and show the classification model

In [95]:
# sample_set_label
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(pca, sample_set_label, test_size=0.3, random_state=101)
# y = [['CEU','FIN','GBR','TSI','YRI'].index(i) for i in sample_set_label]

sample_set_label = sample_set['label']
from sklearn.linear_model import LogisticRegression
predicted = cross_validation.cross_val_predict(LogisticRegression(), pca, sample_set_label, cv=5)
print ("Overall Accuracy :",metrics.accuracy_score(sample_set_label, predicted))
print ("\n")
print (metrics.classification_report(sample_set_label, predicted))

Overall Accuracy : 0.808


             precision    recall  f1-score   support

        CEU       0.92      0.90      0.91        50
        FIN       0.77      0.72      0.74        50
        GBR       0.74      0.80      0.77        50
        TSI       0.68      0.80      0.73        50
        YRI       1.00      0.82      0.90        50

avg / total       0.82      0.81      0.81       250

